In [ ]:
import pandas as pd

In [ ]:
!pip install transformers


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 15.3 MB/s 
     |████████████████████████████████| 141 kB 67.6 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 127 kB 60.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
df = pd.read_csv('/content/berttest12.csv')

In [ ]:
df

,text,label
0,876 f3d 1306,0
1,876 f3d 1307,0
2,876 f3d 1308,0
3,876 f3d 1309,0
4,876 f3d 1310,0
...,...,...
47567,disapprove county v sealy extent appear opinion,1
47568,extent prior opinion state louisiana ralph sla...,1
47569,extent previous opinion o favor louisiana ralp...,1
47570,detail below said court find reverse position ...,1


In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('berttest1.csv')

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ['/content/berttest12.csv'],
                                              'test': 'overruling.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-af3b4d69b93051fa/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 47572
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2400
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/48 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [ ]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
5947/5947 [==============================] - 5538s 929ms/step - loss: 0.3661 - sparse_categorical_accuracy: 0.8305 - val_loss: 0.1691 - val_sparse_categorical_accuracy: 0.9417
Epoch 2/3
5947/5947 [==============================] - 5520s 928ms/step - loss: 0.2353 - sparse_categorical_accuracy: 0.8932 - val_loss: 0.1255 - val_sparse_categorical_accuracy: 0.9550
Epoch 3/3
5947/5947 [==============================] - 5520s 928ms/step - loss: 0.1659 - sparse_categorical_accuracy: 0.9266 - val_loss: 0.1867 - val_sparse_categorical_accuracy: 0.9212


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_227 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred = np.where(model.predict(X_test_pad)>.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
precision_score(y_test, y_pred)

0.88029843


In [ ]:
recall_score(y_test, y_pred)

0.86269351


In [ ]:
f1_score(y_test, y_pred)

0.87140706
